In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import pymysql
import sys
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
import json

database information

In [2]:
# USER = "group13"
# PASSWORD = "123456789"
# HOST = "dublinbikegroup13.c1msfserw61n.us-east-1.rds.amazonaws.com"
# PORT = 3306
# DATABASE = "dbbike13"

HOST = "127.0.0.1"
USER = "root"
PORT = 3306
DATABASE = "dbbike13"
PASSWORD = "qweqweqwe"

connect to database

In [3]:
try:
    engine = create_engine("mysql+pymysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, HOST, PORT, DATABASE), echo=True)
    
except Exception as e:
        sys.exit(e)

In [4]:
df_availability = pd.read_sql_table("availability",engine)
df_weather = pd.read_sql_table("weather", engine)

2023-04-06 16:18:24,607 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-06 16:18:24,609 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-06 16:18:24,610 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-06 16:18:24,610 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-06 16:18:24,610 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-06 16:18:24,611 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-06 16:18:24,612 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-04-06 16:18:24,612 INFO sqlalchemy.engine.Engine [generated in 0.00038s] {'table_schema': 'dbbike13', 'table_name': 'availability'}
2023-04-06 16:18:24,622 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `availability`
2023-04-06 16:18:24,623 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-06 16:18:24,627 INFO sqlalchemy.engine.Engine SELECT availability.number, availability.last_update,

In [5]:
df_availability['datetime_rounded'] = pd.to_datetime(df_availability['last_update']).dt.round('10min')
df_availability

,number,last_update,available_bike_stands,available_bikes,status,datetime_rounded
0,1,2023-03-03 15:16:39,23,8,OPEN,2023-03-03 15:20:00
1,1,2023-03-05 16:03:11,1,29,OPEN,2023-03-05 16:00:00
2,1,2023-03-05 16:08:24,0,30,OPEN,2023-03-05 16:10:00
3,1,2023-03-05 16:49:05,1,29,OPEN,2023-03-05 16:50:00
4,1,2023-03-05 17:11:26,1,29,OPEN,2023-03-05 17:10:00
...,...,...,...,...,...,...
103080,117,2023-04-06 02:00:41,40,0,OPEN,2023-04-06 02:00:00
103081,117,2023-04-06 02:30:54,40,0,OPEN,2023-04-06 02:30:00
103082,117,2023-04-06 10:44:28,31,9,OPEN,2023-04-06 10:40:00
103083,117,2023-04-06 16:06:53,31,9,OPEN,2023-04-06 16:10:00


In [6]:
df_weather['datetime_rounded'] = pd.to_datetime(df_weather['dt']).dt.round('10min')

In [7]:
df_weather

,dt,Clouds,feels_like,humidity,pressure,temp,temp_max,temp_min,sunrise,sunset,visibility,weather_description,weather_main,wind_deg,wind_speed,icon,datetime_rounded
0,2023-03-05 16:41:14,40,279.77,69,1016,282.42,282.80,281.68,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,scattered clouds,Clouds,280,5.14,None,2023-03-05 16:40:00
1,2023-03-05 16:57:03,75,279.55,71,1016,282.08,282.59,281.25,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,270,4.63,None,2023-03-05 17:00:00
2,2023-03-05 17:07:41,75,279.64,71,1015,282.15,282.59,281.25,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,270,4.63,None,2023-03-05 17:10:00
3,2023-03-05 17:15:00,75,279.64,71,1015,282.15,282.59,281.25,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,270,4.63,None,2023-03-05 17:20:00
4,2023-03-05 17:17:09,75,279.03,72,1015,281.98,282.59,280.91,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,280,5.66,None,2023-03-05 17:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,2023-04-05 23:41:39,75,283.34,82,1014,284.05,284.47,283.08,2023-04-05 06:49:45,2023-04-05 20:05:37,10000,broken clouds,Clouds,240,3.60,04n,2023-04-05 23:40:00
1167,2023-04-06 02:06:34,75,278.94,85,1013,281.40,282.25,280.51,2023-04-06 06:47:22,2023-04-06 20:07:25,10000,broken clouds,Clouds,270,4.12,04n,2023-04-06 02:10:00
1168,2023-04-06 02:38:22,75,279.20,83,1013,281.61,281.69,278.68,2023-04-06 06:47:22,2023-04-06 20:07:25,10000,broken clouds,Clouds,270,4.12,04n,2023-04-06 02:40:00
1169,2023-04-06 10:40:02,75,279.00,84,1015,281.96,282.36,281.42,2023-04-06 06:47:22,2023-04-06 20:07:25,10000,broken clouds,Clouds,300,5.66,04d,2023-04-06 10:40:00


In [8]:
df_merged = pd.merge(df_availability, df_weather, on='datetime_rounded')

In [9]:
df_merged

,number,last_update,available_bike_stands,available_bikes,status,datetime_rounded,dt,Clouds,feels_like,humidity,...,temp_max,temp_min,sunrise,sunset,visibility,weather_description,weather_main,wind_deg,wind_speed,icon
0,1,2023-03-05 17:11:26,1,29,OPEN,2023-03-05 17:10:00,2023-03-05 17:07:41,75,279.64,71,...,282.59,281.25,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,270,4.63,None
1,3,2023-03-05 17:08:32,1,19,OPEN,2023-03-05 17:10:00,2023-03-05 17:07:41,75,279.64,71,...,282.59,281.25,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,270,4.63,None
2,4,2023-03-05 17:08:57,0,20,OPEN,2023-03-05 17:10:00,2023-03-05 17:07:41,75,279.64,71,...,282.59,281.25,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,270,4.63,None
3,5,2023-03-05 17:13:10,39,1,OPEN,2023-03-05 17:10:00,2023-03-05 17:07:41,75,279.64,71,...,282.59,281.25,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,270,4.63,None
4,6,2023-03-05 17:12:35,3,17,OPEN,2023-03-05 17:10:00,2023-03-05 17:07:41,75,279.64,71,...,282.59,281.25,2023-03-05 07:04:33,2023-03-05 18:09:05,10000,broken clouds,Clouds,270,4.63,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151614,107,2023-03-25 10:05:15,28,12,OPEN,2023-03-25 10:10:00,2023-03-25 10:06:52,75,282.51,78,...,284.03,282.36,2023-03-25 06:16:23,2023-03-25 18:45:47,10000,broken clouds,Clouds,260,7.20,04d
151615,108,2023-03-25 10:05:47,26,9,OPEN,2023-03-25 10:10:00,2023-03-25 10:06:52,75,282.51,78,...,284.03,282.36,2023-03-25 06:16:23,2023-03-25 18:45:47,10000,broken clouds,Clouds,260,7.20,04d
151616,30,2023-03-23 14:54:35,20,0,OPEN,2023-03-23 14:50:00,2023-03-23 14:53:37,40,285.71,61,...,287.25,285.36,2023-03-23 06:21:15,2023-03-23 18:42:11,10000,scattered clouds,Clouds,210,12.86,03d
151617,42,2023-03-23 14:52:36,28,2,OPEN,2023-03-23 14:50:00,2023-03-23 14:53:37,40,285.71,61,...,287.25,285.36,2023-03-23 06:21:15,2023-03-23 18:42:11,10000,scattered clouds,Clouds,210,12.86,03d


In [10]:
df_whole = df_merged[['number', 'datetime_rounded','available_bike_stands','available_bikes','feels_like','humidity','pressure','weather_main',  'wind_speed' ]]

In [11]:
df_whole

,number,datetime_rounded,available_bike_stands,available_bikes,feels_like,humidity,pressure,weather_main,wind_speed
0,1,2023-03-05 17:10:00,1,29,279.64,71,1015,Clouds,4.63
1,3,2023-03-05 17:10:00,1,19,279.64,71,1015,Clouds,4.63
2,4,2023-03-05 17:10:00,0,20,279.64,71,1015,Clouds,4.63
3,5,2023-03-05 17:10:00,39,1,279.64,71,1015,Clouds,4.63
4,6,2023-03-05 17:10:00,3,17,279.64,71,1015,Clouds,4.63
...,...,...,...,...,...,...,...,...,...
151614,107,2023-03-25 10:10:00,28,12,282.51,78,1002,Clouds,7.20
151615,108,2023-03-25 10:10:00,26,9,282.51,78,1002,Clouds,7.20
151616,30,2023-03-23 14:50:00,20,0,285.71,61,987,Clouds,12.86
151617,42,2023-03-23 14:50:00,28,2,285.71,61,987,Clouds,12.86


In [12]:
df_whole.loc[:, 'day_of_week'] = df_whole['datetime_rounded'].dt.dayofweek
df_whole.loc[:, 'hourly'] = df_whole['datetime_rounded'].dt.hour

C:\Users\Guo.000\AppData\Local\Temp\ipykernel_27012\136827754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole.loc[:, 'day_of_week'] = df_whole['datetime_rounded'].dt.dayofweek
C:\Users\Guo.000\AppData\Local\Temp\ipykernel_27012\136827754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole.loc[:, 'hourly'] = df_whole['datetime_rounded'].dt.hour


In [13]:
print(df_whole['weather_main'].unique())

# Create a LabelEncoder object
le = LabelEncoder()

# Fit and transform the "weather_main" feature
df_whole.loc[:,'weather_main_value'] = le.fit_transform(df_whole['weather_main'])

print(df_whole['weather_main_value'].unique())

['Clouds' 'Drizzle' 'Rain' 'Clear' 'Snow' 'Mist']
[1 2 4 0 5 3]


C:\Users\Guo.000\AppData\Local\Temp\ipykernel_27012\1776046408.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole.loc[:,'weather_main_value'] = le.fit_transform(df_whole['weather_main'])


In [14]:
df_whole['datetime_numeric'] = (df_whole['datetime_rounded'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

C:\Users\Guo.000\AppData\Local\Temp\ipykernel_27012\2278473662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole['datetime_numeric'] = (df_whole['datetime_rounded'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


In [15]:
df_whole.columns = df_whole.columns.astype(str)

In [16]:
df_whole

,number,datetime_rounded,available_bike_stands,available_bikes,feels_like,humidity,pressure,weather_main,wind_speed,day_of_week,hourly,weather_main_value,datetime_numeric
0,1,2023-03-05 17:10:00,1,29,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
1,3,2023-03-05 17:10:00,1,19,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
2,4,2023-03-05 17:10:00,0,20,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
3,5,2023-03-05 17:10:00,39,1,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
4,6,2023-03-05 17:10:00,3,17,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151614,107,2023-03-25 10:10:00,28,12,282.51,78,1002,Clouds,7.20,5,10,1,1679739000
151615,108,2023-03-25 10:10:00,26,9,282.51,78,1002,Clouds,7.20,5,10,1,1679739000
151616,30,2023-03-23 14:50:00,20,0,285.71,61,987,Clouds,12.86,3,14,1,1679583000
151617,42,2023-03-23 14:50:00,28,2,285.71,61,987,Clouds,12.86,3,14,1,1679583000


In [17]:
df_train, df_test = train_test_split(df_whole, test_size=0.3)
print("Train set size:", len(df_train))
print("Test set size:", len(df_test))

Train set size: 106133
Test set size: 45486


In [18]:
# Define the input features and target variables
input_features = ['number', 'feels_like', 'weather_main_value', 'wind_speed','humidity',
                  'pressure','day_of_week', 'hourly']

x_train = df_train[input_features]
y_train = df_train[['available_bike_stands', 'available_bikes']]

x_test = df_test[input_features]
y_test = df_test[['available_bike_stands', 'available_bikes']]

In [19]:
# Create a Random Forest regressor with 100 trees
rf = RandomForestRegressor(n_estimators=100)

# Fit the model on the training data
rf.fit(x_train, y_train)

RandomForestRegressor()

In [20]:
x_test

,number,feels_like,weather_main_value,wind_speed,humidity,pressure,day_of_week,hourly
115369,107,268.85,4,11.32,95,988,3,22
69588,1,271.67,1,6.17,65,992,2,6
91707,109,271.32,1,6.71,84,993,3,0
104383,68,270.65,1,9.26,92,988,3,12
89150,67,271.85,1,7.20,83,992,2,22
...,...,...,...,...,...,...,...,...
64044,48,272.72,0,3.13,71,996,2,0
31968,115,272.26,4,5.14,88,1004,0,20
117466,64,267.99,5,11.32,94,992,4,0
47456,78,271.08,1,3.60,83,1002,1,10


In [21]:
# Make predictions on the test data
y_pred = rf.predict(x_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

print('Mean squared error:', mse)

Mean squared error: 1.8091760643062653


In [22]:
y_pred

array([[13.04      , 26.96      ],
       [19.33257143, 11.66742857],
       [16.        , 13.        ],
       ...,
       [20.0355    , 19.9645    ],
       [26.26      , 13.74      ],
       [39.58      ,  0.42      ]])

In [23]:
y_test

,available_bike_stands,available_bikes
115369,13,27
69588,20,11
91707,16,13
104383,19,21
89150,13,27
...,...,...
64044,23,17
31968,8,22
117466,20,20
47456,25,15


In [24]:
df_whole

,number,datetime_rounded,available_bike_stands,available_bikes,feels_like,humidity,pressure,weather_main,wind_speed,day_of_week,hourly,weather_main_value,datetime_numeric
0,1,2023-03-05 17:10:00,1,29,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
1,3,2023-03-05 17:10:00,1,19,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
2,4,2023-03-05 17:10:00,0,20,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
3,5,2023-03-05 17:10:00,39,1,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
4,6,2023-03-05 17:10:00,3,17,279.64,71,1015,Clouds,4.63,6,17,1,1678036200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151614,107,2023-03-25 10:10:00,28,12,282.51,78,1002,Clouds,7.20,5,10,1,1679739000
151615,108,2023-03-25 10:10:00,26,9,282.51,78,1002,Clouds,7.20,5,10,1,1679739000
151616,30,2023-03-23 14:50:00,20,0,285.71,61,987,Clouds,12.86,3,14,1,1679583000
151617,42,2023-03-23 14:50:00,28,2,285.71,61,987,Clouds,12.86,3,14,1,1679583000


In [25]:
# Define the input features and target variables
input_features = ['feels_like', 'weather_main_value', 'wind_speed','humidity',
                  'pressure','day_of_week', 'hourly']

x_train = df_train[input_features]
y_train = df_train[['available_bike_stands', 'available_bikes']]

x_test = df_test[input_features]
y_test = df_test[['available_bike_stands', 'available_bikes']]

In [26]:
# create list for station model
rf_list = [0] * (len(df_whole['number'].unique())+10)

input_features = ['feels_like', 'weather_main_value', 'wind_speed','humidity',
                  'pressure','day_of_week', 'hourly']

# train for each station
for i in df_whole['number'].unique():
    print(i, end=' ')
    
    x_train = df_train[df_train['number'] == i][input_features]
    y_train = df_train[df_train['number'] == i][['available_bike_stands', 'available_bikes']]
        
    rf = RandomForestRegressor(n_estimators=200)
    rf.fit(x_train, y_train)
    
    rf_list[i] = rf
                

1 3 4 5 6 7 8 9 10 11 12 13 14 15 17 18 20 21 23 24 25 26 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 47 48 49 50 51 52 53 54 55 56 58 59 60 61 62 63 64 65 68 71 72 73 74 75 76 77 80 82 83 84 85 86 87 88 89 90 91 92 96 97 98 99 100 101 102 103 104 105 106 108 111 112 113 114 115 116 117 2 16 19 22 27 28 57 66 67 69 78 79 93 94 95 107 109 110 

In [27]:
len(df_whole['number'].unique())

114

In [28]:
mse_sum = 0
new_train_list = []
for i in df_whole['number'].unique():
    print(i, end=' ')
    
    x_test = df_test[df_test['number'] == i][input_features]                   
    y_test = df_test[df_test['number'] == i][['available_bike_stands', 'available_bikes']]
    
    # Make predictions on the test data
    y_pred = rf_list[i].predict(x_test)
#     print(y_pred.head(5))
    
    # Calculate the mean squared error
    mse = mean_squared_error(y_test, y_pred)
#     print(y_pred.head(5))
    mse_sum += mse

    print('Mean squared error:', mse)
    if mse > 3:
        new_train_list += [i, ]


1 Mean squared error: 1.4338303204870986
3 Mean squared error: 1.4733906426967118
4 Mean squared error: 0.8952038295416008
5 Mean squared error: 7.415776556729684
6 Mean squared error: 0.9704825453260268
7 Mean squared error: 1.036052208578183
8 Mean squared error: 2.286014328556811
9 Mean squared error: 2.296160928086884
10 Mean squared error: 1.4679305916291916
11 Mean squared error: 0.5032182364398479
12 Mean squared error: 0.8696987427872935
13 Mean squared error: 0.48030443682019713
14 Mean squared error: 2.5730269120992757
15 Mean squared error: 1.2198835843401166
17 Mean squared error: 0.3487731214488924
18 Mean squared error: 1.5957693894003082
20 Mean squared error: 2.9211910737155016
21 Mean squared error: 2.0729882375829245
23 Mean squared error: 4.075230448450919
24 Mean squared error: 2.2599514890994037
25 Mean squared error: 0.5196939003365206
26 Mean squared error: 0.7720045527895452
29 Mean squared error: 2.286001359198653
30 Mean squared error: 0.8866196727074409
31 Me

In [29]:
mse_sum / len(df_whole['number'].unique())

1.8410966745435728

In [30]:
for i in new_train_list:
    print(i, end=' ')
    
    x_train = df_train[df_train['number'] == i][input_features]
    y_train = df_train[df_train['number'] == i][['available_bike_stands', 'available_bikes']]
        
    rf = RandomForestRegressor(n_estimators=400)
    rf.fit(x_train, y_train)
    
    rf_list[i] = rf

for i in new_train_list:
    print(i, end=' ')
    
    x_test = df_test[df_test['number'] == i][input_features]                   
    y_test = df_test[df_test['number'] == i][['available_bike_stands', 'available_bikes']]
    
    # Make predictions on the test data
    y_pred = rf_list[i].predict(x_test)
    
    # Calculate the mean squared error
    mse = mean_squared_error(y_test, y_pred)
    print('Mean squared error:', mse)

5 23 33 34 50 56 58 65 68 77 85 117 66 69 5 Mean squared error: 7.258382838991631
23 Mean squared error: 4.032906609647587
33 Mean squared error: 3.2689155955037617
34 Mean squared error: 5.002356675096348
50 Mean squared error: 5.079244983779766
56 Mean squared error: 3.831882495024172
58 Mean squared error: 5.558992648021704
65 Mean squared error: 4.056601726679485
68 Mean squared error: 5.00027572343768
77 Mean squared error: 3.837817316891007
85 Mean squared error: 3.1927404409693216
117 Mean squared error: 4.6237861344976965
66 Mean squared error: 3.3235244279980343
69 Mean squared error: 3.6785608885648746


In [31]:
import pickle

for i in df_whole['number'].unique():
    with open('station_'+str(i)+'.pkl','wb') as handle:
        pickle.dump(rf_list[i], handle, pickle.HIGHEST_PROTOCOL)

In [39]:
y_pred = rf_list[13].predict([[285.27, 1, 5.14, 55, 1017, 3, 16]])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [40]:
y_pred

array([[21.79827381,  8.19172619]])